In [1]:
# import necessary dependecies 
import re # regular expression to check if theres a pattern 
import tiktoken # Implementing Byte Pair Encoding using exisitng open-source library 

import torch 
from torch.utils.data import Dataset, DataLoader 

In [ ]:
# Check if Mac supports PyTorch Acceleration with Apple Silicon Chip 
print(torch.backends.mps.is_available())

In [3]:
class SimpleTokenizer: 
    def __init__(self, vocab):
        self.string_to_integer = vocab
        self.integer_to_string = {i:s for s,i in vocab.items()} 
        
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)  # Split text by specific punctuation and whitespace
        preprocessed = [item.strip() for item in preprocessed if item.strip()]  # Strip and filter out empty items
        preprocessed = [item if item in self.string_to_integer else "<|unk|>" for item in preprocessed]

        ids = [self.string_to_integer[s] for s in preprocessed if s in self.string_to_integer]  # Map the text to ids
        return ids

    def decode(self, ids): 
        text = " ".join([self.integer_to_string[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    

In [4]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride): 
        self.input_ids = []
        self.target_ids = [] 
        
        token_ids = tokenizer.encode(txt, allowed_special = {"|endoftext|>"}) 
        
        for i in range(0, len(token_ids) - max_length, stride): # Creating context window 
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk)) # Creates a input matricies  
            self.target_ids.append(torch.tensor(target_chunk)) # Creates a input matricies  
            
    def __len__(self): 
        return len(self.input_ids)
    
    def __getitem__(self, idx): 
        return self.input_ids[idx], self.target_ids[idx]


In [5]:
def create_dataloader_v1(txt, batch_size =4, max_length=256, stride=128, shuffle=True, drop_last=True, num_worker = 0):
    tokenizer = tiktoken.get_encoding("gpt2") 
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle,
        drop_last=drop_last, 
        num_workers=num_worker)
    
    return dataloader

In [ ]:
with open("The_Verdict.txt", "r", encoding="utf=8") as f: 
    text = f.read() 
    
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
preprocessed = [item for item in result if item.strip()]                #further remove whitespaces from string list 
all_tokens = sorted(list(set(preprocessed)))               # set() builds a hash set, and list() makes the set into a list. 
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

vocab


In [9]:
# Test - Instantiate an object 
tokenizer = SimpleTokenizer(vocab) 
ids = tokenizer.encode(text) 

decoded_text = tokenizer.decode(ids) 


In [ ]:
ids

In [ ]:
decoded_text

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

In [ ]:
# Test end of text token in the case of additional test source
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

In [ ]:
tokenizer = SimpleTokenizer(vocab)
print(tokenizer.encode(text))


In [ ]:
print(tokenizer.decode(tokenizer.encode(text)))

# TEST - Byte Pair Encoding 

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2") 
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

In [ ]:
print(type(integers))

In [ ]:
text = "AKwirw ier"
integers = tokenizer.encode(text)
tokenizer.decode(integers)

In [22]:
with open("The_Verdict.txt", 'r', encoding="utf=8") as f:
    raw_text = f.read()
    
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
first_batch

 

[tensor([[  464,  4643, 11600,   628]]),
 tensor([[ 4643, 11600,   628,   198]])]

In [20]:
second_batch = next(data_iter)

In [23]:
second_batch # Each stride is a representation of the next context window size

[tensor([[ 4643, 11600,   628,   198]]),
 tensor([[11600,   628,   198,   197]])]

In [26]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter) 
print("Inputs:\n", inputs) 
print("\nTargets:\n", targets)

Inputs:
 tensor([[  464,  4643, 11600,   628],
        [  198,   197,   197,   197],
        [  197,   197,  7407,   342],
        [  854, 41328,   628,   628],
        [  198,   198,  1129,  2919],
        [  628,   628,   198,   198],
        [ 3109,  9213,   422, 11145],
        [  271,  1668,   319,  2693]])

Targets:
 tensor([[ 4643, 11600,   628,   198],
        [  197,   197,   197,   197],
        [  197,  7407,   342,   854],
        [41328,   628,   628,   198],
        [  198,  1129,  2919,   628],
        [  628,   198,   198,  3109],
        [ 9213,   422, 11145,   271],
        [ 1668,   319,  2693,  1467]])
